In [8]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/barcoded_flu_pdmH1N1/lib/python3.8/site-packages', '/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\x86 \x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c_results/viral_tags_bcs_in_cells/scProgenyProduction_trial3_cell_barcodes_with_viral_tags.csv.gz\x94\x8cLresults/viral_fastq10x/scProgenyProduction_trial3_viral_genes_by_cell.csv.gz\x94\x8c?results/pacbio/scProgenyProduction_trial3_consensus_gene.csv.gz\x94\x8cOresults/viral_fastq10x/scProgenyProduction_trial3_viral_bc_by_cell_valid.csv.gz\x94\x8cQresults/viral_progeny/scProgenyProduction_trial3_filtered_progeny_viral_bc.csv.gz\x94\x8cTresults/viral_fastq10x/scProgenyProduction_trial3_contributes_progeny_by_cell.csv.gz\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x10cell_annotations\x94K\x00N\x86\x94\x8c\x17viral_genes_by_cell_csv\x94K\x01N\x86\x94\x8c\x19pacbio_consensus_gene_csv\x94K\x02N\x86\x94\x8c\x18viral_barcodes_valid_csv\x94K\x03N\x86\x94\x8c\x1dfiltered_progeny_viral_bc_csv\x94K\x04N\x86\x94\x8c\x1fcontributes_progeny_by_cell_csv\x94K\x05N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh!\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\')}\x94\x8c\x05_name\x94h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bh\x13h\nh\x15h\x0bh\x17h\x0ch\x19h\rh\x1bh\x0eh\x1dh\x0fub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8cDresults/viral_fastq10x/scProgenyProduction_trial3_integrate_data.csv\x94a}\x94(h\x11}\x94\x8c\x13integrated_data_csv\x94K\x00N\x86\x94sh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bh8h5ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94]\x94(\x8c\x05fluHA\x94\x8c\x05fluNA\x94ea}\x94(h\x11}\x94\x8c\x14barcoded_viral_genes\x94K\x00N\x86\x94sh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bhLhGub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x1ascProgenyProduction_trial3\x94a}\x94(h\x11}\x94\x8c\x04expt\x94K\x00N\x86\x94sh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94b\x8c\x04expt\x94h[ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01e}\x94(h\x11}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94uh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bhqK\x01hsK\x01ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c<results/logs/integrate_data_scProgenyProduction_trial3.ipynb\x94a}\x94(h\x11}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bh\x85h\x82ub\x8c\x06config\x94}\x94(\x8c\x08max_cpus\x94K\x10\x8c\x07log_dir\x94\x8c\x0cresults/logs\x94\x8c\x0cfastq10x_dir\x94\x8c\x10results/fastq10x\x94\x8c\npacbio_dir\x94\x8c\x0eresults/pacbio\x94\x8c\x0emkfastq10x_dir\x94\x8c\x1fresults/fastq10x/mkfastq_output\x94\x8c\ngenome_dir\x94\x8c\x0fresults/genomes\x94\x8c\trefgenome\x94\x8c\x19results/genomes/refgenome\x94\x8c\x14aligned_fastq10x_dir\x94\x8c\x18results/aligned_fastq10x\x94\x8c\x12viral_fastq10x_dir\x94\x8c\x16results/viral_fastq10x\x94\x8c\x1bviral_tags_bcs_in_cells_dir\x94\x8c\x1fresults/viral_tags_bcs_in_cells\x94\x8c\x11viral_progeny_dir\x94\x8c\x15results/viral_progeny\x94\x8c\x13align_and_parse_dir\x94\x8c#results/pacbio/CA09_align_and_parse\x94\x8c\x0fcell_genome_ftp\x94\x8cmftp://ftp.ensembl.org/pub/release-98/fasta/canis_familiaris/dna/Canis_familiaris.CanFam3.1.dna.toplevel.fa.gz\x94\x8c\x0ccell_gtf_ftp\x94\x8c^ftp://ftp.ensembl.org/pub/release-98/gtf/canis_familiaris/Canis_familiaris.CanFam3.1.98.gtf.gz\x94\x8c\x0cviral_genome\x94\x8c!data/flu_sequences/flu-CA09.fasta\x94\x8c\tviral_gtf\x94\x8c\x1fdata/flu_sequences/flu-CA09.gtf\x94\x8c\rviral_genbank\x94\x8c\x1edata/flu_sequences/flu-CA09.gb\x94\x8c\x14viral_tag_identities\x94\x8c+data/flu_sequences/flu-CA09_viral_tags.yaml\x94\x8c\x0fviral_amplicons\x94\x8c.data/flu_sequences/pacbio_amplicons/fluCA09.gb\x94\x8c\x11amplicon_features\x94\x8c9data/flu_sequences/pacbio_amplicons/fluCA09_features.yaml\x94\x8c\x1eamplicon_linearization_primers\x94\x8c=data/flu_sequences/pacbio_amplicons/linearization_primers.tsv\x94\x8c\x15amplicon_to_reference\x94\x8c=data/flu_sequences/pacbio_amplicons/amplicon_to_reference.csv\x94\x8c\x0escoreGapNoncan\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapGCAG\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapATAC\x94J\xfc\xff\xff\xff\x8c\x14cb_whitelist_10x_url\x94\x8cvhttps://raw.githubusercontent.com/10XGenomics/cellranger/master/lib/python/cellranger/barcodes/3M-february-2018.txt.gz\x94\x8c\x10cb_whitelist_10x\x94\x8c-results/aligned_fastq10x/cb_whitelist_10x.txt\x94\x8c\ncb_len_10x\x94K\x10\x8c\x0bumi_len_10x\x94K\x0c\x8c\x14total_UMI_deviations\x94G@\x04\x00\x00\x00\x00\x00\x00\x8c\x13infection_threshold\x94G?\x84z\xe1G\xae\x14{\x8c\rviral_tag_fdr\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x1dviral_barcode_upstream_length\x94K\x1c\x8c\x16viral_barcode_mismatch\x94K\x00\x8c\x12viral_barcode_minq\x94K\x1e\x8c\x17progeny_detection_limit\x94\x8c\x041e-5\x94\x8c\x0cviral_bc_fdr\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x0eccs_min_length\x94K2\x8c\x0eccs_max_length\x94M\x88\x13\x8c\nccs_min_rq\x94G?\xef\xf7\xce\xd9\x16\x87+\x8c\x0bexperiments\x94}\x94\x8c\x1ascProgenyProduction_trial3\x94}\x94(\x8c\x0bdescription\x94X\x1b\x01\x00\x00Single-cell transcriptomics performed on even lower MOI-infected sample. Innoculum volumes based on `scProgenyProduction_trial1` and `scProgenyProduction_trial2` data. Loaded all collected cells from two 24-well wells, each infected with either wildtype or dblySn viral tag variants.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-N4laiIAy3AUePJdig7wR\x94\x8c\rexpect_ncells\x94M\x10\'\x8c\x0ftranscriptomics\x94}\x94(\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe5\x03\x0c\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210312_D00300_1192_BHKHNYBCX3/raw/210312_D00300_1192_BHKHNYBCX3/\x94\x8c\x04lane\x94K\x01\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xdfC\x04\x07\xe5\x04\t\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210409_D00300_1210_BHKC7KBCX3/raw/210409_D00300_1210_BHKC7KBCX3/\x94\x8c\x04lane\x94\x8c\x01*\x94\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xdfC\x04\x07\xe5\x08\x19\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210825_D00300_1310_BHMH5TBCX3/raw/210825_D00300_1310_BHMH5TBCX3/\x94\x8c\x04lane\x94h\xf4\x8c\x10index_sequencing\x94\x8c\x04none\x94uh\xdfC\x04\x07\xe5\t\x0f\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8cY/shared/ngs/illumina/bloom_lab/210915_VH00699_2_AAAMCL7M5/raw/210915_VH00699_2_AAAMCL7M5/\x94\x8c\x04lane\x94h\xf4\x8c\x10index_sequencing\x94\x8c\x04none\x94uu\x8c\x17pacbio_viral_sequencing\x94}\x94(\x8c\x1c2021-08-05_all_segments_run1\x94\x8cp/fh/fast/bloom_j/SR/ngs/pacbio/210730_SequelIIe/r64272e_20210730_182729/1_A01/m64272e_210730_193026.subreads.bam\x94\x8c\x1c2021-09-17_all_segments_run2\x94\x8cp/fh/fast/bloom_j/SR/ngs/pacbio/210917_SequelIIe/r64272e_20210917_174421/1_A01/m64272e_210917_175514.subreads.bam\x94u\x8c\x0eviral_barcodes\x94}\x94(\x8c\x0bsupernatant\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluHA-rep1_S9_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluHA-rep2_S13_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluNA-rep1_S1_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-sup-fluNA-rep2_S1_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluHA-rep1_S10_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluHA-rep2_S14_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluNA-rep1_S2_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluNA-rep2_S6_L001_R1_001.fastq.gz\x94suuu\x8c\x10second_infection\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x85/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluHA-rep1_S2_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x85/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluHA-rep2_S3_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluNA-rep1_S11_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluNA-rep2_S12_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x89/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluHA-rep1_S4_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x89/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluHA-rep2_S5_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluNA-rep1_S13_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdfC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluNA-rep2_S14_L001_R1_001.fastq.gz\x94suuuuusu\x8c\x04rule\x94\x8c\x0eintegrate_data\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c[/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir('/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing');
######## snakemake preamble end #########


# Integrate data
This notebook takes processed data and integrates it into a single dataframe.

The general structure of the dataframe is that each cell barcode is listed on a row, and features of that cell are listed in columns. Cells that have more than one valid viral barcode identified may have multiple rows--one for each valid viral barcode.

Import python modules:


In [9]:
from IPython.display import display

import pandas as pd

Variables:

In [10]:
# Input sources
cell_annotations_csv = snakemake.input.cell_annotations
viral_genes_by_cell_csv = snakemake.input.viral_genes_by_cell_csv
pacbio_consensus_gene_csv = snakemake.input.pacbio_consensus_gene_csv
viral_barcodes_valid_csv = snakemake.input.viral_barcodes_valid_csv
filtered_progeny_viral_bc_csv = snakemake.input.filtered_progeny_viral_bc_csv
contributes_progeny_by_cell_csv = snakemake.input.contributes_progeny_by_cell_csv

# # Params and wildcards
barcoded_viral_genes = snakemake.params.barcoded_viral_genes
expt = snakemake.wildcards.expt

# Output
integrated_data_csv = snakemake.output.integrated_data_csv

## Load data and transform into wide format
Load data from the following sources:  
* `cell_annotations_csv` contains a list of all cell barcodes, their infection status/tag, total_UMIs, viral_UMIs, and frac_viral_UMIs  

* `viral_genes_by_cell_csv` contains every cell barcode, each viral gene, whether the viral gene is present, the fraction of UMIs from that gene, and the total number of viral genes detected in that cell  

* `viral_barcodes_valid_csv` contains each infected cell, the valid viral barcodes detected in that cell, viral_bc_UMIs, and frac_viral_bc_UMIs  

* `filtered_progeny_viral_bc_csv` contains the progeny source, the tag of the progeny source, the valid viral barcodes detected in that progeny source, and the frequency of the viral barcodes in that progeny source  

* `pacbio_consensus_gene_csv` contains the cell barcode, each viral gene, and the consensus genotype of the viral gene in that cell

Some of these dataframes are in a long format, with many rows per cell barcode. I will transform them into a wide format, so that each cell barcode has only one row. The only exception is the viral barcodes, which will have one row per cell barcode-viral barcode pair.

Load cell barcodes, tags, and basic metrics for every cell in transcriptome:

In [11]:
cell_annotations = pd.read_csv(cell_annotations_csv)
cell_annotations = (
    cell_annotations
    [['cell_barcode',
      'infected',
      'infecting_viral_tag',
      'total_UMIs',
      'viral_UMIs',
      'frac_viral_UMIs']])

display(cell_annotations)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429
...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,uninfected,none,44890,4,0.000089
3125,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048
3126,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182
3127,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486


Load viral genes detected in each cell:

In [12]:
viral_genes = pd.read_csv(viral_genes_by_cell_csv)
viral_genes = viral_genes.rename(columns={'gene_UMIs': 'transcriptome_UMIs',
                                          'frac_gene_UMIs': 'frac_UMIs',
                                          'gene_present': 'present'})
viral_genes = (viral_genes
               .pivot(
    index=['cell_barcode', 'n_viral_genes'],
    columns=['gene'],
    values=['transcriptome_UMIs', 'frac_UMIs', 'present']))
viral_genes.columns = ['_'.join(col).strip() for col in viral_genes.columns.values]
viral_genes = viral_genes.reset_index()
display(viral_genes)

,cell_barcode,n_viral_genes,transcriptome_UMIs_fluHA,transcriptome_UMIs_fluM,transcriptome_UMIs_fluNA,transcriptome_UMIs_fluNP,transcriptome_UMIs_fluNS,transcriptome_UMIs_fluPA,transcriptome_UMIs_fluPB1,transcriptome_UMIs_fluPB2,...,frac_UMIs_fluPB1,frac_UMIs_fluPB2,present_fluHA,present_fluM,present_fluNA,present_fluNP,present_fluNS,present_fluPA,present_fluPB1,present_fluPB2
0,AAACCCAGTAACAAGT,0,2,3,0,0,1,0,0,0,...,0,0,False,False,False,False,False,False,False,False
1,AAACCCATCATTGCTT,1,0,5,0,3,1,0,0,1,...,0,1.10971e-05,False,False,False,False,False,False,False,True
2,AAACGAAAGATGTTGA,0,3,10,0,1,4,0,0,0,...,0,0,False,False,False,False,False,False,False,False
3,AAACGAAGTACTTCCC,7,2,13060,420,4139,5772,23,506,160,...,0.00890406,0.00281551,False,True,True,True,True,True,True,True
4,AAACGAAGTAGACGTG,8,319,1582,31,608,2055,11,45,3,...,0.000361908,2.41272e-05,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,0,0,4,0,0,0,0,0,0,...,0,0,False,False,False,False,False,False,False,False
3125,TTTGATCTCCCGTTCA,0,0,1,0,1,1,0,0,0,...,0,0,False,False,False,False,False,False,False,False
3126,TTTGGAGAGTTGCCTA,1,0,6,1,1,4,0,0,0,...,0,0,False,False,True,False,False,False,False,False
3127,TTTGGAGGTATCGTTG,8,93,1668,45,418,1230,7,57,8,...,0.000379671,5.32872e-05,True,True,True,True,True,True,True,True


Load viral gene genotypes:

In [13]:
viral_genotypes = pd.read_csv(pacbio_consensus_gene_csv)
viral_genotypes = viral_genotypes.rename(columns={'total_UMI': 'pacbio_UMIs',
                                                  'consensus_mutations_annotated': 'mutations',
                                                  'support': 'mutation_support'})
viral_genotypes = (
    viral_genotypes
    .pivot(index=['cell_barcode'],
           columns=['gene'],
           values=['mutations', 'pacbio_UMIs', 'mutation_support']))
viral_genotypes.columns = ['_'.join(col).strip() for col in viral_genotypes.columns.values]
viral_genotypes = viral_genotypes.reset_index()
display(viral_genotypes)

,cell_barcode,mutations_fluHA,mutations_fluM,mutations_fluNA,mutations_fluNP,mutations_fluNS,mutations_fluPA,mutations_fluPB1,mutations_fluPB2,pacbio_UMIs_fluHA,...,pacbio_UMIs_fluPB1,pacbio_UMIs_fluPB2,mutation_support_fluHA,mutation_support_fluM,mutation_support_fluNA,mutation_support_fluNP,mutation_support_fluNS,mutation_support_fluPA,mutation_support_fluPB1,mutation_support_fluPB2
0,AAACGAAGTACTTCCC,NaN,WT,WT,WT,WT,WT,WT,WT,NaN,...,46,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAACGAAGTAGACGTG,WT,WT,WT,WT,WT,WT,WT,NaN,36,...,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAAGGATTCTGATGGT,WT,WT,WT,WT,WT,WT,WT,NaN,4,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAGGGCTCCGCACTT,WT,WT,WT,WT,WT,WT,WT,del168to2007 del2097to2106,11,...,5,1611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1597/1611 1601/1611
4,AAAGTCCAGTAGAGTT,WT,WT,WT,WT,WT,WT,WT,WT,19,...,88,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,TTGGGATGTAACGATA,WT,WT,WT,WT,WT,WT,WT,WT,36,...,28,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,TTGGGATTCCGATTAG,WT,WT,WT,WT,WT,WT,WT,WT,30,...,18,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,TTTACCAGTCGCTTAA,WT,WT,WT,WT,WT,NaN,WT,NaN,8,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,TTTCACAAGCCAAGCA,T465A_H155Q,WT,WT,WT,NaN,WT,NaN,WT,90,...,NaN,104,85/90,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load valid viral barcodes in each infected cell:

In [14]:
transcriptome_viral_barcodes = pd.read_csv(viral_barcodes_valid_csv)
transcriptome_viral_barcodes = transcriptome_viral_barcodes.drop(columns=['valid_viral_bc'])
transcriptome_viral_barcodes = transcriptome_viral_barcodes.rename(columns={'gene': 'barcoded_gene'})
assert set(transcriptome_viral_barcodes['barcoded_gene']) == set(barcoded_viral_genes), \
       "Barcoded genes in barcode counts do not match expectation."
display(transcriptome_viral_barcodes)

,cell_barcode,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs
0,AAACGAAGTAGACGTG,fluHA,AAGTAAGCGACATGAG,251,0.002019
1,AAAGGATTCTGATGGT,fluHA,GTGGAGTCGCCAGTTC,114,0.001424
2,AAAGGGCTCCGCACTT,fluHA,AATCTACGAGGGAAAC,33,0.000257
3,AAAGGGCTCCGCACTT,fluHA,ATGGATCAGATTTCCT,10,0.000078
4,AAAGGGCTCCGCACTT,fluHA,GCAAAAATACAAAGTA,28,0.000218
...,...,...,...,...,...
895,TTGGGATTCCGATTAG,fluNA,GTTTCGTGAGCATGTA,25,0.000182
896,TTTACCAGTCGCTTAA,fluNA,ATTTGGATGTAGGGGC,4,0.000035
897,TTTACCAGTCGCTTAA,fluNA,TTGGAGGAGACCCGTG,7,0.000061
898,TTTCACAAGCCAAGCA,fluNA,GGTATCAGTTATTGTT,186,0.002718


Load progeny viral barcode frequencies:

In [15]:
progeny_viral_barcodes = pd.read_csv(filtered_progeny_viral_bc_csv)
progeny_viral_barcodes = progeny_viral_barcodes.drop(columns=['Unnamed: 0'])
progeny_viral_barcodes = (progeny_viral_barcodes
                          .rename(columns={'tag': 'infecting_viral_tag',
                                           'average_freq': 'freq',
                                           'gene': 'barcoded_gene'}))
assert set(progeny_viral_barcodes['barcoded_gene']) == set(barcoded_viral_genes), \
       "Barcoded genes in barcode counts do not match expectation."
progeny_viral_barcodes = (
    progeny_viral_barcodes
    .pivot(index=['viral_barcode', 'infecting_viral_tag', 'barcoded_gene'],
           columns=['source'],
           values=['freq']))
progeny_viral_barcodes.columns = ['_'.join(col).strip() for col in progeny_viral_barcodes.columns.values]
progeny_viral_barcodes = progeny_viral_barcodes.reset_index()
display(progeny_viral_barcodes)

,viral_barcode,infecting_viral_tag,barcoded_gene,freq_second_infection,freq_supernatant
0,AAAAACACGTTCTATA,wt,fluNA,0.000010,0.000010
1,AAAACATGATGACGCC,wt,fluNA,0.000010,0.000010
2,AAAACTAGTTAGAGCA,wt,fluHA,0.000010,0.000010
3,AAAAGTTCTTGGATGT,wt,fluNA,0.000010,0.000010
4,AAAATTGTCCAATATA,syn,fluHA,0.000010,0.000010
...,...,...,...,...,...
790,TTTCGTGAATCACTGC,wt,fluHA,0.000010,0.000073
791,TTTCTTTACTCAGAAT,wt,fluNA,0.003833,0.002060
792,TTTGCACACGTCTAAT,wt,fluHA,0.000010,0.000010
793,TTTTAACGTTATACTA,wt,fluNA,0.000010,0.000010


Load whether each infected cell contributes any progeny:

In [16]:
contributes_progeny = pd.read_csv(contributes_progeny_by_cell_csv)
contributes_progeny = contributes_progeny.rename(columns={'gene': 'barcoded_gene',
                                                          'max_progeny_freq': 'max_freq'})
assert set(transcriptome_viral_barcodes['barcoded_gene']) == set(barcoded_viral_genes), \
       "Barcoded genes in barcode counts do not match expectation."
contributes_progeny = (
    contributes_progeny
    .pivot(index=['cell_barcode'],
           columns=['source'],
           values=['max_freq', 'contributes_progeny']))
contributes_progeny.columns = ['_'.join(col).strip() for col in contributes_progeny.columns.values]
contributes_progeny = contributes_progeny.reset_index()
display(contributes_progeny)

,cell_barcode,max_freq_second_infection,max_freq_supernatant,contributes_progeny_second_infection,contributes_progeny_supernatant
0,AAACGAAGTAGACGTG,1e-05,7.62399e-05,False,True
1,AAAGGATTCTGATGGT,0.077933,0.0131834,True,True
2,AAAGGGCTCCGCACTT,0.000124415,0.00195048,True,True
3,AAAGTCCAGTAGAGTT,1e-05,1e-05,False,False
4,AACCAACAGGGTCTTT,2.17873e-05,0.000971284,True,True
...,...,...,...,...,...
354,TTGGGATGTAACGATA,0.000153675,0.00128394,True,True
355,TTGGGATTCCGATTAG,1e-05,1e-05,False,False
356,TTTACCAGTCGCTTAA,1.8581e-05,0.00121169,True,True
357,TTTCACAAGCCAAGCA,1e-05,1e-05,False,False


## Integrate data into single dataframe

Start with `cell_annotations` dataframe. Bring in viral gene expression information.

In [17]:
integrated_df = pd.merge(
    left=cell_annotations,
    right=viral_genes,
    on='cell_barcode',
    how='outer',
    validate='one_to_one'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,transcriptome_UMIs_fluHA,transcriptome_UMIs_fluM,transcriptome_UMIs_fluNA,...,frac_UMIs_fluPB1,frac_UMIs_fluPB2,present_fluHA,present_fluM,present_fluNA,present_fluNP,present_fluNS,present_fluPA,present_fluPB1,present_fluPB2
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,2,3,0,...,0,0,False,False,False,False,False,False,False,False
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5,0,...,0,1.10971e-05,False,False,False,False,False,False,False,True
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,3,10,0,...,0,0,False,False,False,False,False,False,False,False
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,2,13060,420,...,0.00890406,0.00281551,False,True,True,True,True,True,True,True
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,319,1582,31,...,0.000361908,2.41272e-05,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,uninfected,none,44890,4,0.000089,0,0,4,0,...,0,0,False,False,False,False,False,False,False,False
3125,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1,0,...,0,0,False,False,False,False,False,False,False,False
3126,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,6,1,...,0,0,False,False,True,False,False,False,False,False
3127,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,0.000379671,5.32872e-05,True,True,True,True,True,True,True,True


Bring in viral gene genotypes. Many cells do not have pacbio data. For all missing data, fill all counts with 0 and fill all mutations with "Not Detected".

In [19]:
integrated_df = pd.merge(
    left=integrated_df,
    right=viral_genotypes,
    on='cell_barcode',
    how='left',
    validate='one_to_one'
)

# Fill empty mutations columns with "Not Detected"
integrated_df[[col for col in integrated_df if col.startswith('mutations')]] = (
     integrated_df[[col for col in integrated_df if col.startswith('mutations')]]
     .fillna('Not Detected'))
# Fill empty pacbio_UMIs columns with 0
integrated_df[[col for col in integrated_df if col.startswith('pacbio_UMIs')]] = (
     integrated_df[[col for col in integrated_df if col.startswith('pacbio_UMIs')]]
     .fillna(0))
# Fill NaN support columns with "Not Applicable"
integrated_df[[col for col in integrated_df if col.startswith('mutation_support')]] = (
     integrated_df[[col for col in integrated_df if col.startswith('mutation_support')]]
     .fillna('Not Applicable'))

display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,transcriptome_UMIs_fluHA,transcriptome_UMIs_fluM,transcriptome_UMIs_fluNA,...,pacbio_UMIs_fluPB1_y,pacbio_UMIs_fluPB2_y,mutation_support_fluHA_y,mutation_support_fluM_y,mutation_support_fluNA_y,mutation_support_fluNP_y,mutation_support_fluNS_y,mutation_support_fluPA_y,mutation_support_fluPB1_y,mutation_support_fluPB2_y
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,2,3,0,...,0,0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5,0,...,0,0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,3,10,0,...,0,0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,2,13060,420,...,46,21,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,319,1582,31,...,15,0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,uninfected,none,44890,4,0.000089,0,0,4,0,...,0,0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
3125,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1,0,...,0,0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
3126,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,6,1,...,0,0,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
3127,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,23,7,Not Applicable,Not Applicable,31/31,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable


Bring in valid viral barcodes for cells that have them:

In [20]:
integrated_df = pd.merge(
    left=integrated_df,
    right=transcriptome_viral_barcodes,
    on='cell_barcode',
    how='left',
    validate='one_to_many'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,transcriptome_UMIs_fluHA,transcriptome_UMIs_fluM,transcriptome_UMIs_fluNA,...,mutation_support_fluNA_y,mutation_support_fluNP_y,mutation_support_fluNS_y,mutation_support_fluPA_y,mutation_support_fluPB1_y,mutation_support_fluPB2_y,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,2,3,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,3,10,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,2,13060,420,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,319,1582,31,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,fluHA,AAGTAAGCGACATGAG,251.0,0.002019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN
3678,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,6,1,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN
3679,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,31/31,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,fluHA,AGTAAACTTCCTCGCG,65.0,0.000433
3680,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,31/31,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,fluNA,ACATCTTATTTACACG,39.0,0.000260


Bring in progeny frequencies:

In [21]:
integrated_df = pd.merge(
    left=integrated_df,
    right=progeny_viral_barcodes,
    on=['infecting_viral_tag', 'barcoded_gene', 'viral_barcode'],
    how='left',
    validate='many_to_one'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,transcriptome_UMIs_fluHA,transcriptome_UMIs_fluM,transcriptome_UMIs_fluNA,...,mutation_support_fluNS_y,mutation_support_fluPA_y,mutation_support_fluPB1_y,mutation_support_fluPB2_y,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs,freq_second_infection,freq_supernatant
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,2,3,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,3,10,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,2,13060,420,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,319,1582,31,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,fluHA,AAGTAAGCGACATGAG,251.0,0.002019,0.000010,0.000076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1,0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN
3678,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,6,1,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN
3679,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,fluHA,AGTAAACTTCCTCGCG,65.0,0.000433,0.000010,0.000695
3680,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,fluNA,ACATCTTATTTACACG,39.0,0.000260,0.000028,0.001790


Bring in annotation of whether each infected cell contributes any progeny:

In [22]:
integrated_df = pd.merge(
    left=integrated_df,
    right=contributes_progeny,
    on=['cell_barcode'],
    how='left',
    validate='many_to_one'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,transcriptome_UMIs_fluHA,transcriptome_UMIs_fluM,transcriptome_UMIs_fluNA,...,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs,freq_second_infection,freq_supernatant,max_freq_second_infection,max_freq_supernatant,contributes_progeny_second_infection,contributes_progeny_supernatant
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,2,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,3,10,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,2,13060,420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,319,1582,31,...,fluHA,AAGTAAGCGACATGAG,251.0,0.002019,0.000010,0.000076,1e-05,7.62399e-05,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3678,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,6,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3679,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,fluHA,AGTAAACTTCCTCGCG,65.0,0.000433,0.000010,0.000695,2.81273e-05,0.00179022,True,True
3680,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,93,1668,45,...,fluNA,ACATCTTATTTACACG,39.0,0.000260,0.000028,0.001790,2.81273e-05,0.00179022,True,True


## Check integrated dataframe

Check that the number of total cells has not changed.

In [23]:
assert integrated_df['cell_barcode'].nunique() == \
    len(cell_annotations['cell_barcode']), \
    "Total number of cells changed"

Check that every viral barcode has a `barcoded_gene`, `frac_viral_bc_UMIS`, `freq_supernatant`, and `freq_second_infection`.

In [24]:
assert len(integrated_df.query('viral_barcode.notnull()', engine='python')) == \
    len(integrated_df.query('barcoded_gene.notnull()', engine='python')) == \
    len(integrated_df.query('frac_viral_bc_UMIs.notnull()', engine='python')) == \
    len(integrated_df.query('freq_supernatant.notnull()', engine='python')) == \
    len(integrated_df.query('freq_second_infection.notnull()', engine='python')), \
    "Mismatch in viral barcode data."

## Output integrated dataframe

In [25]:
# save CSV
integrated_df.to_csv(integrated_data_csv, index=False)